In [ ]:

import pandas as pd
import numpy as np
from axisfuzzy.analysis.pipeline import FuzzyPipeline
from axisfuzzy.analysis.component.basic import (
    ToolNormalization, ToolWeightNormalization, ToolStatistics, ToolSimpleAggregation
)
from axisfuzzy.analysis.build_in import ContractCrispTable, ContractWeightVector, ContractNormalizedWeights
from example_basic_dependency_file import (ContractAccuracyVector,
                                            ContractOrderResult,
                                            ContractMultiMetrics,
                                            DemoDataGenerator,
                                            DemoScoreCalculator,
                                            DemoMultiOutputAnalyzer,
                                            DemoRanker,
                                            DemoDataAggregator)
from examples.analysis.example_components_demo import main_pipeline

In [ ]:
def create_data_preprocessing_pipeline_fixed():
    """
    修正后的数据预处理子管道

    重要修改：确保输出是 ContractCrispTable 而不是 ContractStatisticsDict

    Pipeline: raw_data -> normalization -> normalized_data (ContractCrispTable)

    Returns
    -------
    FuzzyPipeline
        A reusable data preprocessing pipeline that outputs normalized data.
    """
    # Create components
    normalizer = ToolNormalization(method='min_max', axis=0)

    # Build sub-pipeline
    preprocessing_pipeline = FuzzyPipeline(name="DataPreprocessing_SubPipeline")

    # Define inputs
    raw_data = preprocessing_pipeline.input("raw_data", contract=ContractCrispTable)

    # Processing steps - 注意：这里只做标准化，不计算统计量
    # 这样输出仍然是 ContractCrispTable 类型
    normalized_data = preprocessing_pipeline.add(normalizer.run, data=raw_data)

    # 移除统计计算步骤，以保持输出为 ContractCrispTable
    return preprocessing_pipeline

In [ ]:
def create_data_analysis_pipeline():
    """
    创建独立的数据分析子管道（用于统计分析）

    Pipeline: raw_data -> statistics -> stats_dict (ContractStatisticsDict)

    Returns
    -------
    FuzzyPipeline
        A pipeline dedicated to statistical analysis.
    """
    # Create components
    stats_calculator = ToolStatistics(axis=1)

    # Build sub-pipeline
    analysis_pipeline = FuzzyPipeline(name="DataAnalysis_SubPipeline")

    # Define inputs
    input_data = analysis_pipeline.input("input_data", contract=ContractCrispTable)

    # Processing steps
    data_stats = analysis_pipeline.add(stats_calculator.run, data=input_data)

    return analysis_pipeline

In [ ]:
def create_scoring_pipeline_fixed():
    """
    修正后的评分子管道（保持不变，因为合约已经正确）

    Pipeline: {processed_data, weights} -> score_calculation -> ranking -> {scores, rankings}

    Returns
    -------
    FuzzyPipeline
        A reusable scoring and ranking pipeline.
    """
    # Create components
    score_calculator = DemoScoreCalculator(score_method='weighted_sum')
    ranker = DemoRanker(ascending=False)

    # Build sub-pipeline
    scoring_pipeline = FuzzyPipeline(name="Scoring_SubPipeline")

    # Define inputs
    processed_data = scoring_pipeline.input("processed_data", contract=ContractCrispTable)
    weights = scoring_pipeline.input("weights", contract=ContractNormalizedWeights)

    # Processing steps
    scores = scoring_pipeline.add(score_calculator.run, data=processed_data, weights=weights)
    rankings = scoring_pipeline.add(ranker.run, scores=scores)

    return scoring_pipeline

In [ ]:
def demo_simple_nested_pipeline_fixed():
    """
    修正后的简单嵌套管道演示

    Main Pipeline: raw_data -> [DataPreprocessing SubPipeline] -> [Scoring SubPipeline] -> final_results
    """
    print("=" * 80)
    print("Demo: Fixed Simple Nested Pipeline (2-level nesting)")
    print("=" * 80)

    # Create sub-pipelines - 使用修正后的版本
    preprocessing_sub = create_data_preprocessing_pipeline_fixed()
    scoring_sub = create_scoring_pipeline_fixed()

    # Create main pipeline
    main_pipeline = FuzzyPipeline(name="Main_NestedPipeline_Fixed")

    # Define main inputs
    raw_data = main_pipeline.input("raw_data", contract=ContractCrispTable)
    raw_weights = main_pipeline.input("raw_weights", contract=ContractWeightVector)

    # Add weight normalization
    weight_normalizer = ToolWeightNormalization()
    normalized_weights = main_pipeline.add(weight_normalizer.run, weights=raw_weights)

    # Add nested preprocessing pipeline
    # 现在 preprocessing_result 是 ContractCrispTable 类型
    preprocessing_result = main_pipeline.add(
        preprocessing_sub,
        raw_data=raw_data
    )

    # Add nested scoring pipeline
    # 合约匹配：ContractCrispTable -> ContractCrispTable ✓
    final_result = main_pipeline.add(
        scoring_sub,
        processed_data=preprocessing_result,
        weights=normalized_weights
    )

    # Test data
    test_data = pd.DataFrame({
        'Criterion1': [0.8, 0.6, 0.9, 0.7, 0.85],
        'Criterion2': [0.7, 0.8, 0.6, 0.9, 0.75],
        'Criterion3': [0.9, 0.7, 0.8, 0.6, 0.80]
    }, index=['Alt1', 'Alt2', 'Alt3', 'Alt4', 'Alt5'])

    test_weights = np.array([0.4, 0.35, 0.25])

    # Run nested pipeline
    result = main_pipeline.run({
        "raw_data": test_data,
        "raw_weights": test_weights
    })

    print(f"Input data shape: {test_data.shape}")
    print(f"Input weights: {test_weights}")
    print(f"Final result type: {type(result)}")
    print("\nFinal Results:")
    print(result)

    return main_pipeline, result

In [ ]:
def demo_comprehensive_nested_pipeline():
    """
    展示如何同时使用预处理和分析管道的完整演示

    Main Pipeline:
    - Branch 1: raw_data -> preprocessing -> scoring
    - Branch 2: raw_data -> analysis -> statistics
    """
    print("\n" + "=" * 80)
    print("Demo: Comprehensive Nested Pipeline with Analysis Branch")
    print("=" * 80)

    # Create all sub-pipelines
    preprocessing_sub = create_data_preprocessing_pipeline_fixed()
    scoring_sub = create_scoring_pipeline_fixed()
    analysis_sub = create_data_analysis_pipeline()

    # Create main pipeline
    main_pipeline = FuzzyPipeline(name="Comprehensive_NestedPipeline")

    # Define main inputs
    raw_data = main_pipeline.input("raw_data", contract=ContractCrispTable)
    raw_weights = main_pipeline.input("raw_weights", contract=ContractWeightVector)

    # Weight normalization
    weight_normalizer = ToolWeightNormalization()
    normalized_weights = main_pipeline.add(weight_normalizer.run, weights=raw_weights)

    # Branch 1: Preprocessing + Scoring
    preprocessed_data = main_pipeline.add(
        preprocessing_sub,
        raw_data=raw_data
    )

    scoring_results = main_pipeline.add(
        scoring_sub,
        processed_data=preprocessed_data,
        weights=normalized_weights
    )

    # Branch 2: Statistical Analysis (独立分支)
    statistical_results = main_pipeline.add(
        analysis_sub,
        input_data=raw_data  # 直接使用原始数据进行统计分析
    )

    # Test data
    test_data = pd.DataFrame({
        'Performance': [85, 78, 92, 88, 75],
        'Quality': [90, 85, 88, 92, 80],
        'Cost': [70, 95, 85, 75, 90]
    }, index=['Product_A', 'Product_B', 'Product_C', 'Product_D', 'Product_E'])

    test_weights = np.array([0.5, 0.3, 0.2])

    # Run comprehensive nested pipeline
    result = main_pipeline.run({
        "raw_data": test_data,
        "raw_weights": test_weights
    })

    print(f"Input data shape: {test_data.shape}")
    print(f"Input weights: {test_weights}")
    print(f"Final result type: {type(result)}")
    print("\nComprehensive Results:")
    if isinstance(result, dict):
        for key, value in result.items():
            print(f"\n{key}:")
            print(f"  Type: {type(value)}")
            print(f"  Value: {value}")
    else:
        print(result)

    return main_pipeline, result

In [ ]:
main_pipeline1, result1 = demo_simple_nested_pipeline_fixed()
main_pipeline2, result2 = demo_comprehensive_nested_pipeline()

In [ ]:
main_pipeline1.visualize()

In [ ]:
main_pipeline2.visualize()

In [ ]:
result1

In [ ]:
result2

In [ ]:
def create_level3_data_cleaning_pipeline():
    """
    Level 3: 基础数据清洗管道

    Pipeline: raw_data -> outlier_removal -> cleaned_data
    """
    cleaning_pipeline = FuzzyPipeline(name="Level3_DataCleaning")

    # 输入
    raw_data = cleaning_pipeline.input("raw_data", contract=ContractCrispTable)

    # 简单的数据清洗（使用标准化作为清洗步骤的示例）
    normalizer = ToolNormalization(method='z_score', axis=0)
    cleaned_data = cleaning_pipeline.add(normalizer.run, data=raw_data)

    return cleaning_pipeline


def create_level3_feature_engineering_pipeline():
    """
    Level 3: 特征工程管道

    Pipeline: cleaned_data -> feature_transformation -> engineered_features
    """
    feature_pipeline = FuzzyPipeline(name="Level3_FeatureEngineering")

    # 输入
    cleaned_data = feature_pipeline.input("cleaned_data", contract=ContractCrispTable)

    # 特征工程（使用聚合作为特征工程的示例）
    aggregator = ToolSimpleAggregation(operation='mean', axis=1)
    feature_scores = feature_pipeline.add(aggregator.run, data=cleaned_data)

    # 将聚合结果转换回DataFrame格式以保持合约兼容性
    def convert_to_dataframe(scores):
        """将Series转换为单列DataFrame"""
        if isinstance(scores, pd.Series):
            return pd.DataFrame({'aggregated_feature': scores})
        return scores

    # 标记为合约方法
    convert_to_dataframe._is_contract_method = True
    convert_to_dataframe._contract_inputs = {'scores': 'ContractWeightVector'}
    convert_to_dataframe._contract_outputs = {'output': 'ContractCrispTable'}

    engineered_features = feature_pipeline.add(convert_to_dataframe, scores=feature_scores)

    return feature_pipeline


def create_level2_preprocessing_pipeline():
    """
    Level 2: 综合预处理管道（嵌套Level 3管道）

    Pipeline: raw_data -> [Level3_DataCleaning] -> [Level3_FeatureEngineering] -> processed_data
    """
    preprocessing_pipeline = FuzzyPipeline(name="Level2_Preprocessing")

    # 输入
    raw_data = preprocessing_pipeline.input("raw_data", contract=ContractCrispTable)

    # 嵌套Level 3管道
    cleaning_sub = create_level3_data_cleaning_pipeline()
    feature_sub = create_level3_feature_engineering_pipeline()

    # 数据清洗
    cleaned_data = preprocessing_pipeline.add(
        cleaning_sub,
        raw_data=raw_data
    )

    # 特征工程
    processed_data = preprocessing_pipeline.add(
        feature_sub,
        cleaned_data=cleaned_data
    )

    return preprocessing_pipeline


def create_level2_analysis_pipeline():
    """
    Level 2: 综合分析管道

    Pipeline: processed_data -> multi_analysis -> analysis_results
    """
    analysis_pipeline = FuzzyPipeline(name="Level2_Analysis")

    # 输入
    processed_data = analysis_pipeline.input("processed_data", contract=ContractCrispTable)

    # 多维度分析
    multi_analyzer = DemoMultiOutputAnalyzer()
    analysis_results = analysis_pipeline.add(multi_analyzer.run, data=processed_data)

    return analysis_pipeline


def create_level1_master_pipeline():
    """
    Level 1: 主控管道（嵌套Level 2管道）

    Complex 3-Level Nested Pipeline:
    raw_data -> [Level2_Preprocessing] -> [Level2_Analysis] -> final_results
    """
    master_pipeline = FuzzyPipeline(name="Level1_Master_ComplexNested")

    # 输入
    raw_data = master_pipeline.input("raw_data", contract=ContractCrispTable)

    # 嵌套Level 2管道
    preprocessing_sub = create_level2_preprocessing_pipeline()
    analysis_sub = create_level2_analysis_pipeline()

    # 预处理阶段
    processed_data = master_pipeline.add(
        preprocessing_sub,
        raw_data=raw_data
    )

    # 分析阶段
    final_results = master_pipeline.add(
        analysis_sub,
        processed_data=processed_data
    )

    return master_pipeline


def demo_complex_nested_pipeline():
    """
    演示复杂的3层嵌套管道
    """
    print("=" * 80)
    print("Demo: Complex 3-Level Nested Pipeline")
    print("=" * 80)

    # 创建复杂嵌套管道
    master_pipeline = create_level1_master_pipeline()

    # 测试数据
    test_data = pd.DataFrame({
        'Metric_A': [85.2, 78.1, 92.5, 88.7, 75.3, 91.2, 83.8],
        'Metric_B': [90.1, 85.5, 88.9, 92.3, 80.7, 87.6, 89.4],
        'Metric_C': [70.5, 95.2, 85.1, 75.8, 90.3, 82.7, 88.1],
        'Metric_D': [88.3, 76.9, 91.7, 85.4, 87.8, 79.5, 92.0]
    }, index=['Entity_1', 'Entity_2', 'Entity_3', 'Entity_4', 'Entity_5', 'Entity_6', 'Entity_7'])

    print(f"Input Data Shape: {test_data.shape}")
    print("Input Data Preview:")
    print(test_data.head(3))

    # 执行复杂嵌套管道
    try:
        result = master_pipeline.run({"raw_data": test_data})

        print(f"\nComplex Nested Pipeline Result Type: {type(result)}")
        print("Final Results:")
        if isinstance(result, dict):
            for key, value in result.items():
                print(f"\n{key}:")
                print(f"  Type: {type(value)}")
                if hasattr(value, 'shape'):
                    print(f"  Shape: {value.shape}")
                print(f"  Content: {value}")
        else:
            print(result)

        # 展示管道结构
        print(f"\nPipeline Structure:")
        print(f"Total Steps: {len(master_pipeline.steps)}")
        print(f"Input Nodes: {list(master_pipeline.input_nodes.keys())}")

        return master_pipeline, result

    except Exception as e:
        print(f"Error executing complex nested pipeline: {e}")
        raise


def demo_deeply_nested_pipeline():
    """
    演示极深层嵌套管道（4层嵌套）
    """
    print("\n" + "=" * 80)
    print("Demo: Deeply Nested Pipeline (4-Level)")
    print("=" * 80)

    # Level 4: 原子操作管道
    def create_level4_atomic_operations():
        atomic_pipeline = FuzzyPipeline(name="Level4_AtomicOps")

        input_data = atomic_pipeline.input("input_data", contract=ContractCrispTable)

        # 最基础的操作
        normalizer = ToolNormalization(method='min_max', axis=1)
        normalized = atomic_pipeline.add(normalizer.run, data=input_data)

        return atomic_pipeline

    # Level 3: 组合Level 4操作
    def create_level3_composite():
        composite_pipeline = FuzzyPipeline(name="Level3_Composite")

        input_data = composite_pipeline.input("input_data", contract=ContractCrispTable)

        atomic_sub = create_level4_atomic_operations()
        processed = composite_pipeline.add(atomic_sub, input_data=input_data)

        return composite_pipeline

    # Level 2: 组合Level 3操作
    def create_level2_integrated():
        integrated_pipeline = FuzzyPipeline(name="Level2_Integrated")

        input_data = integrated_pipeline.input("input_data", contract=ContractCrispTable)

        composite_sub = create_level3_composite()
        result = integrated_pipeline.add(composite_sub, input_data=input_data)

        return integrated_pipeline

    # Level 1: 主控管道
    deeply_nested_pipeline = FuzzyPipeline(name="Level1_DeeplyNested")

    raw_data = deeply_nested_pipeline.input("raw_data", contract=ContractCrispTable)

    integrated_sub = create_level2_integrated()
    final_result = deeply_nested_pipeline.add(integrated_sub, input_data=raw_data)

    # 测试数据
    test_data = pd.DataFrame(np.random.rand(4, 3) * 100,
                           columns=['Dim1', 'Dim2', 'Dim3'],
                           index=['Sample1', 'Sample2', 'Sample3', 'Sample4'])

    print(f"Input Data for 4-Level Nesting:")
    print(test_data)

    # 执行4层嵌套管道
    result = deeply_nested_pipeline.run({"raw_data": test_data})

    print(f"\nDeeply Nested Pipeline Result:")
    print(f"Type: {type(result)}")
    print(f"Shape: {result.shape if hasattr(result, 'shape') else 'N/A'}")
    print(result)

    return deeply_nested_pipeline, result

In [ ]:
cleaning_pipeline = create_level3_data_cleaning_pipeline()
feature_pipeline = create_level3_feature_engineering_pipeline()
preprocessing_pipeline = create_level2_preprocessing_pipeline()
analysis_pipeline = create_level2_analysis_pipeline()
master_pipeline = create_level1_master_pipeline()
complex_pipeline, result_master = demo_complex_nested_pipeline()
deeply_nested_pipeline, result_nested = demo_deeply_nested_pipeline()

In [ ]:
# cleaning_pipeline.visualize()
# feature_pipeline.visualize()
# preprocessing_pipeline.visualize()
# analysis_pipeline.visualize()
# master_pipeline.visualize()
# complex_pipeline.visualize()
# deeply_nested_pipeline.visualize()

In [ ]:
complex_pipeline.steps